<a href="https://colab.research.google.com/github/okadatsuneo/GBFS_Import/blob/main/gbfstocsv_docomo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ドコモ・バイクシェアのstation_information.jsonファイルからOSMにインポートするためのcsvファイルを作成する。

In [1]:
#モジュールのインポート
import pandas as pd
import json
from pathlib import Path

In [2]:
#ドライブのマウントはセッションが切れると解除される。ファイルも削除されるので、MyDrive下に置く。
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Googleドライブにあるstation_information.jsonを開く
# ファイルパスの結合はpathlibを使用
path = Path('/content/drive/MyDrive/python/')
file_name_station_info = 'd_station_information.json'
file_path = path / file_name_station_info
with file_path.open() as f:
    json_dict = json.load(f)

#'stations'の値を新たなリストlist_stationとする。
list_station = json_dict['data']['stations']

#list_stationの内容をDataFrameにする。
df = pd.DataFrame(list_station)

#'ref:gbfs'の列を"<system_id>:<station_id>"の形で作成する。
df['ref:gbfs'] = 'docomo-cycle:' + df['station_id']

#出力に必要な列だけ選択する:df3
df3 = df[['lat','lon', 'name', 'ref:gbfs', 'capacity']].copy()

#df3にOSM用の固定値を追加
df3['amenity'] = 'bicycle_rental'
df3['brand'] = 'Docomo Bike Share'
df3['brand:ja'] = 'ドコモ・バイクシェア'
df3['brand:wikidata'] = 'Q55533296'
df3['website'] = 'https://docomo-cycle.jp/'
df3['authentication:app'] = 'yes'
df3['fee'] = 'yes'

#範囲が日本全国に渡るので9つのエリアに分割してインポートできるようエリア情報を追加する。
# 緯度経度の範囲の中で要素数が同じになるように分割し(qcut)、ラベルを列の値として追加する。
df3['lat_label'] = pd.qcut(df3['lat'], 3, labels=False)
df3['lon_label'] = pd.qcut(df3['lon'], 3, labels=False)

#ファイル出力
df3.to_csv(path / 'd_stations.csv')
print("処理が完了しました。")


処理が完了しました。
